In [1]:
! pip install Pillow
! pip install numpy
! pip install match

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ------------------------------------- 267.9/267.9 kB 16.1 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB ? eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB ? eta 0:00:00
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Created wheel for match: filename=match-0.3.2-py3-none-any.whl size=9326 sha256=e0ee6199e9015ea6e9a89ba98437685fc63f78baceeb1cf8f4ca329454bd1c4b
  Stored in directory: c:\users\hyejun\appdata\local\pip\cache\wheels\ce\e9\98\868ca1cf4b2dd2de9e6ee99231fb687f2e3dc725e7b6736234
Successfully built match


## Gaussian Filtering

In [2]:
from PIL import Image
import numpy as np
import math

### 1. boxfilter

In [3]:
def boxfilter(n):
    # n이 odd가 아니면 assert error
    assert n%2==1, "Dimension must be odd"
    return np.ones((n, n)) / (n * n)


In [4]:
boxfilter(3)

array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]])

In [5]:
boxfilter(4)

AssertionError: Dimension must be odd

In [6]:
boxfilter(5)

array([[0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04]])

### 2.gauss1d(sigma)

In [7]:
def gauss1d(sigma):
    length = math.ceil(sigma * 6)
    if length % 2 == 0:
        length += 1
    xs = np.arange(-(length // 2), length // 2 + 1, 1)
    gaussian = np.exp(-xs**2 / (2 * sigma ** 2))
    return gaussian / np.sum(gaussian)

In [8]:
gauss1d(0.3)

array([0.00383626, 0.99232748, 0.00383626])

In [9]:
gauss1d(0.5)

array([0.10650698, 0.78698604, 0.10650698])

In [10]:
gauss1d(1)

array([0.00443305, 0.05400558, 0.24203623, 0.39905028, 0.24203623,
       0.05400558, 0.00443305])

In [11]:
gauss1d(2)

array([0.0022182 , 0.00877313, 0.02702316, 0.06482519, 0.12110939,
       0.17621312, 0.19967563, 0.17621312, 0.12110939, 0.06482519,
       0.02702316, 0.00877313, 0.0022182 ])

### 3. gauss2d(sigma)

In [12]:
def gauss2d(sigma):
    gaussian1d = gauss1d(sigma)
    return np.outer(gaussian1d, gaussian1d)

In [13]:
gauss2d(0.5)

array([[0.01134374, 0.08381951, 0.01134374],
       [0.08381951, 0.61934703, 0.08381951],
       [0.01134374, 0.08381951, 0.01134374]])

In [14]:
gauss2d(1)

array([[1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [1.76900911e-03, 2.15509428e-02, 9.65846250e-02, 1.59241126e-01,
        9.65846250e-02, 2.15509428e-02, 1.76900911e-03],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05]])

### 4. convovle2d(array, filter)

In [15]:
def convolve2d(array, filter):
    height, width = array.shape
    filterSize = filter.shape[0]

    paddingSize = filterSize // 2

    modifiedArray = np.zeros((height + 2 * paddingSize, width + 2 * paddingSize))
    modifiedArray[paddingSize:-paddingSize, paddingSize: -paddingSize] = array

    result = np.zeros((height, width))

    np.flip(filter, axis=0)
    np.flip(filter, axis=1)

    for i in range(height):
        for j in range(width):
            window = modifiedArray[i:i+filterSize, j:j+filterSize]

            result[i,j] = np.sum(window * filter)
    
    return result
    

In [16]:
def gaussconvolve2d(array, sigma):
    filter = gauss2d(sigma)
    return convolve2d(array, filter)

In [17]:
im = Image.open('./images/2b_dog.bmp')
im.show()
im = im.convert('L')
im = np.asarray(im)

im = gaussconvolve2d(im, 3)
im = np.clip(im, 0, 255).astype(np.uint8)
im = Image.fromarray(im)
im.save("./result_images/dog_convolution.png", "PNG")
im.show()

## Hypbrid Images

### 1. Gaussian filtered low frequency image

In [19]:
tower = Image.open('./images/3b_tower.bmp')
tower = np.asarray(tower)
sigma = 3

r = gaussconvolve2d(tower[:,:,0], sigma)
g = gaussconvolve2d(tower[:,:,1], sigma)
b = gaussconvolve2d(tower[:,:,2], sigma)

tower = np.dstack([r, g, b])
tower = np.clip(tower, 0, 255).astype(np.uint8)
towerImage = Image.fromarray(tower)
towerImage.save("./result_images/blur_image.png", "PNG")
towerImage.show()

In [20]:
eiffel = Image.open('./images/3a_eiffel.bmp')
eiffel = np.asarray(eiffel)
sigma = 3

r = gaussconvolve2d(eiffel[:,:,0], sigma)
g = gaussconvolve2d(eiffel[:,:,1], sigma)
b = gaussconvolve2d(eiffel[:,:,2], sigma)

blurredEiffel = np.dstack([r, g, b])
blurredEiffel = np.clip(blurredEiffel, 0, 255).astype(np.int16)

highFreqEiffel = eiffel - blurredEiffel

highFreqEiffel = np.clip(highFreqEiffel, -128, 127).astype(np.int16)

visulalizeHighFreqEiffel = highFreqEiffel + np.ones_like(highFreqEiffel) * 128
visulalizeHighFreqEiffel = np.clip(visulalizeHighFreqEiffel, 0, 255).astype(np.uint8)
visulalizeHighFreqEiffel = Image.fromarray(visulalizeHighFreqEiffel)

visulalizeHighFreqEiffel.save("./result_images/hig_freq_image.png", "PNG")
visulalizeHighFreqEiffel.show()

In [21]:
hybridImage = tower + highFreqEiffel
hybridImage = np.clip(hybridImage, 0, 255).astype(np.uint8)
hybridImage = Image.fromarray(hybridImage)
hybridImage.save("./result_images/hybird_image.png", "PNG")
hybridImage.show()